# TV Minimization


In this example $Y=AX+N$ simulation enviroment will be created.
In this equation:
- Y is the observation
- A is system geometry parameters
- X is varible (the thing which is observed )
- N is randomly distrubuted zero mean gaussian noise

Size of this vector and matrices are:
$$ Y_{mx1}=A_{mxn}X_{nx1} + N_{mx1} $$


now contiune with code.


## 1. Configuration

### Import libraries


In [ ]:
import numpy as np
import os
from math import sqrt
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline 

plt.rcParams['figure.figsize'] = [16, 10]
plt.gray()


### Helper functions

In [ ]:

def normalize(a):
    return a/255

def denormalize(a):
    return a*255

# image to matrix
def i2m(a):
    return np.array(a)/255

# matrix to image
def m2i(a):
    return Image.fromarray(a*255).convert("L")

def rmse(X1, X2):
    return np.sqrt(np.mean((X1-X2)**2))


### Params

In [ ]:
EPS = 0.0000000000000001 # epsilon
BETA =  0.2 #
LEARNING_RATE = 0.001

## 2. Init

To simulate the problem the following equation is defined

$ Y = X + N $

where:
- X is the real & non-noisy image ( It is the varible which should be estimated )
- N is random gaussion noise
- Y is the observation


In [ ]:
#Xreal = i2m(  Image.open('test/zebra.jpg').convert('L'))#.resize((256,256), Image.ANTIALIAS))#.rotate(-90) )
Xreal = i2m(  Image.open('test/lines.jpg').convert('L'))
pixelSizeX, pixelSizeY = np.shape(Xreal)
noise = np.random.rand(pixelSizeX, pixelSizeY) * 0.2

Y = Xreal + noise

In [ ]:
m2i(Y)

## 3. TV

### Define TV

Tv

$
T V_{2 D}(X)=\sum_{i}^{K} \sum_{j}^{L} \sqrt{\left(X_{i, j}-X_{i-1, j}\right)^{2}+\left(X_{i, j}-X_{i, j-1}\right)^{2}}
$

Gradient of tv

$Grad(X)=\frac{2\left(X_{i, j}-X_{i-1, j}\right)+2\left(X_{i j}-X_{i j-1}\right)}{\sqrt{\left(X_{i, j}-X_{i-1, j}\right)^{2}+\left(X_{i, j}-X_{i, j-1}\right)^{2}+\varepsilon}}
-\frac{2\left(X_{i+1, j}-X_{i, j}\right)}{\sqrt{\left(X_{i+1, j}-X_{i, j}\right)^{2}+\left(X_{i+1, j}-X_{i+1, j-1}\right)^{2}+\varepsilon}}
-\frac{2\left(X_{i, j+1}-X_{i, j}\right)}{\sqrt{\left(X_{i, j+1}-X_{i, j}\right)^{2}+\left(X_{i, j+1}-X_{i-1, j+1}\right)^{2}+\varepsilon}}
$

### Classic TV

In [ ]:
def tvNorm(x):
    """Computes the total variation norm and its gradient. From jcjohnson/cnn-vis."""
    x_diff = x - np.roll(x, -1, axis=1)
    y_diff = x - np.roll(x, -1, axis=0)
    grad_norm2 = x_diff**2 + y_diff**2 + EPS
    norm = np.sum(np.sqrt(grad_norm2))
    return norm

def tvGrad(x):
    """Computes the total variation norm and its gradient. From jcjohnson/cnn-vis."""
    x_diff = x - np.roll(x, -1, axis=1)
    y_diff = x - np.roll(x, -1, axis=0)
    grad_norm2 = x_diff**2 + y_diff**2 + EPS
    dgrad_norm = 0.5 / np.sqrt(grad_norm2)
    dx_diff = 2 * x_diff * dgrad_norm
    dy_diff = 2 * y_diff * dgrad_norm
    grad = dx_diff + dy_diff
    grad[:, 1:] -= dx_diff[:, :-1]
    grad[1:, :] -= dy_diff[:-1, :]
    return grad


def l2Norm(x):
    """Computes 1/2 the square of the L2-norm and its gradient."""
    return np.sum(x**2)

def l2NormGrad(x):
    return -2 * x


### Directional TV

In [ ]:
def directionalTVNormM(X):
    M,N = np.shape(X)
    XM = X
    tv = 0
    for i in range(3, M-3):
        for j in range(3, N-3):
            payda1 = (XM[i,j] - XM[i+1,j])**2
            payda2 = (XM[i,j] - XM[i,j+1])**2
            payda3 = (XM[i,j-1] - XM[i,j+2])**2
            tv += sqrt( payda1 + payda2 + payda3 + EPS)
    return tv

def directionalTVNorm(x):
    payda1 = x - np.roll(x, +1, axis=0)
    payda2 = x - np.roll(x, +1, axis=1)
    payda3 = np.roll(x, -1, axis=1) - np.roll(x, +2, axis=1)
    return np.sum( np.sqrt( payda1**2 + payda2**2 + payda3**2) )


def gradMA(X):
    M,N = np.shape(X)
    XM = X
    gradX = np.zeros((M,N))
    # XM(4:end-3,4:end-3)=X
    for i in range(3, M-3):
        for j in range(3, N-3):
            pay = 0.5 * -2 * ( XM[i,j-3] - XM[i,j] )
            payda1 = (XM[i,j-2] - XM[i+1,j-2])**2
            payda2 = (XM[i,j-2] - XM[i,j-1])**2
            payda3 = (XM[i,j-3] - XM[i,j])**2
            payda = sqrt( payda1 + payda2 + payda3 + EPS)
            gradX[i,j] = pay / payda
    return gradX

def gradA(x):
    pay = -1 * ( x - np.roll(x, -3, axis=1) )
    payda1 = np.roll(x, -2, axis=1) - np.roll(np.roll(x, 1, axis=0), -2, axis=1 )
    payda2 = np.roll(x, -2, axis=1) - np.roll(x, -1, axis=1)
    payda3 = np.roll(x, -3, axis=1) - x
    payda = np.sqrt( payda1**2 + payda2**2 + payda3**2 + EPS )
    gradX = pay / payda
    return gradX
    

def gradMB(X):
    M,N = np.shape(X)
    XM = X
    gradX = np.zeros((M,N))
    # XM(4:end-3,4:end-3)=X

    for i in range(3, M-3):
        for j in range(3, N-3):
            pay = 0.5 * -2 * ( XM[i,j-1] - XM[i,j] )
            payda1 = (XM[i,j-1] - XM[i+1,j-1])**2
            payda2 = (XM[i,j-1] - XM[i,j])**2
            payda3 = (XM[i,j-2] - XM[i,j+1])**2
            payda = sqrt( payda1 + payda2 + payda3 + EPS)
            gradX[i,j] = pay / payda
    return gradX

def gradB(x):
    pay = -1 * ( np.roll(x, -1, axis=1) -  x )
    payda1 = np.roll(x, -1, axis=1) - np.roll(np.roll(x, 1, axis=0), 1, axis=1 )
    payda2 = np.roll(x, -1, axis=1) - x
    payda3 = np.roll(x, -2, axis=1) - np.roll(x, +1, axis=1)
    payda = np.sqrt( payda1**2 + payda2**2 + payda3**2 + EPS )
    gradX = pay / payda
    return gradX


def gradMC(X):
    M,N = np.shape(X)
    XM = X
    gradX = np.zeros((M,N))
    for i in range(3, M-3):
        for j in range(3, N-3):
            pay = 0.5 * 2 * ( XM[i,j] - XM[i+1,j] + XM[i,j] - XM[i,j+1] )
            payda1 = (XM[i,j] - XM[i+1,j])**2
            payda2 = (XM[i,j] - XM[i,j+1])**2
            payda3 = (XM[i,j-1] - XM[i,j+2])**2
            payda = sqrt( payda1 + payda2 + payda3 + EPS)
            gradX[i,j] = pay / payda
    return gradX

def gradC(x):
    pay =  2 * x - np.roll(x, +1, axis=0) - np.roll(x, +1, axis=1)
    payda1 = x - np.roll(x, +1, axis=0)
    payda2 = x - np.roll(x, +1, axis=1)
    payda3 = np.roll(x, -1, axis=1) - np.roll(x, +2, axis=1)
    payda = np.sqrt( payda1**2 + payda2**2 + payda3**2 + EPS )
    gradX = pay / payda
    return gradX



def gradMD(X):
    M,N = np.shape(X)
    XM = X
    gradX = np.zeros((M,N))
    for i in range(3, M-3):
        for j in range(3, N-3):
            pay = 0.5 * 2 * ( XM[i,j] - XM[i,j+3] )
            payda1 = (XM[i,j+1] - XM[i+1,j+1])**2
            payda2 = (XM[i,j+1] - XM[i,j+2])**2
            payda3 = (XM[i,j] - XM[i,j+3])**2
            payda = sqrt( payda1 + payda2 + payda3 + EPS)
            gradX[i,j] = pay / payda
    return gradX


def gradD(x):
    pay = x - np.roll(x, +3, axis=1)
    payda1 = np.roll(x, +1, axis=1) - np.roll(np.roll(x, +1, axis=0), +1, axis=1 )
    payda2 = np.roll(x, +1, axis=1) - np.roll(x, +2, axis=1)
    payda3 = x - np.roll(x, +3, axis=1)
    payda = np.sqrt( payda1**2 + payda2**2 + payda3**2 + EPS )
    gradX = pay / payda
    return gradX

def gradME(X):
    M,N = np.shape(X)
    XM = X
    gradX = np.zeros((M,N))
    # XM(4:end-3,4:end-3)=X

    for i in range(3, M-3):
        for j in range(3, N-3):
            pay = 0.5 * -2 * ( XM[i-1,j] - XM[i,j] )
            payda1 = (XM[i-1,j] - XM[i,j])**2
            payda2 = (XM[i-1,j] - XM[i-1,j+1])**2
            payda3 = (XM[i-1,j-1] - XM[i-1,j+2])**2
            payda = sqrt( payda1 + payda2 + payda3 + EPS)
            gradX[i,j] = pay / payda
    return gradX

def gradE(x):
    pay = -1 * ( np.roll(x, -1, axis=0) - x )
    payda1 = np.roll(x, -1, axis=0) - x
    payda2 = np.roll(x, -1, axis=0) - np.roll(np.roll(x, -1, axis=0), +1, axis=1 )
    payda3 = np.roll(np.roll(x, -1, axis=0), -1, axis=1 ) - np.roll(np.roll(x, -1, axis=0), +2, axis=1 )
    payda = np.sqrt( payda1**2 + payda2**2 + payda3**2 + EPS )
    gradX = pay / payda
    return gradX


def directionalTVGradM(X):
    return gradMA(X)+gradMB(X)+gradMC(X)+gradMD(X)+gradME(X)

def directionalTVGrad(X):
    return gradA(X)+gradB(X)+gradC(X)+gradD(X)+gradE(X)



In [ ]:
plt.imshow( directionalTVGrad(Xreal))
plt.figure()
#plt.imshow( gradMA(Xreal) )
#(gradA(Xreal) - gradMA(Xreal)).max()

In [ ]:
%timeit directionalTVGrad(Xreal)
%timeit directionalTVNorm(Xreal)

## 4. Cost functions

### Classic TV

### Define Cost Function

$\hat{X}=\underset{X}{\arg \min }\left[\|Y- X\|_{2}+\beta T V(X)\right]$

In [ ]:
def tvCost(Y, X, beta):
    return l2Norm(Y-X) + beta * tvNorm(X)

def tvCostGrad(Y, X, beta):
    return l2NormGrad(Y-X) + beta * tvGrad(X)

### Directional TV

In [ ]:
def directionalTVCost(Y, X, beta):
    return l2Norm(Y-X) + beta * directionalTVNorm(X)

def directionalTVCostGrad(Y, X, beta):
    return l2NormGrad(Y-X) + beta * directionalTVGrad(X)

In [ ]:
plt.rcParams['figure.figsize'] = [16, 10]

fig = plt.figure()

ax = fig.add_subplot(121)
ax.title.set_text('Original Image Variation on X Axis')
ax.imshow(m2i(Xreal-np.roll(Xreal, -1, axis=1)))

ax = fig.add_subplot(122)
ax.title.set_text('Noisy Image Variation on X Axis')
ax.imshow(m2i(Y-np.roll(Y, -1, axis=1)))

plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = [16, 10]

fig = plt.figure()

ax = fig.add_subplot(121)
ax.title.set_text('Original Image')
ax.imshow(m2i(directionalTVGrad(Xreal)))

ax = fig.add_subplot(122)
ax.title.set_text('Noisy Image')
ax.imshow(m2i(directionalTVGrad(Y)))

plt.show()

## 4. Minimization

### Gradient Descent

Generic form of gradient descent.

$X^{k+1} = X^k - \alpha \nabla Cost$

Gradient descent method for tv

$X^{k+1} = X^k - \alpha \nabla( |Y- X\|_{2}+\beta T V(X) ) $


### Sum all up for TV

In [ ]:
def minimize(costFunction=None, costFunctionGrad=None, \
             real=None, initial=None, \
             beta=0.3, learningrate=0.001, maxIter=1000, finishCondition=10):
    costHistory = []
    rmseHistory = [] 
    Xk = initial
    for i in range(maxIter):
        if i%30 == 0:
            print(i, costFunction(initial, Xk, beta))
        Xnext = Xk - learningrate * costFunctionGrad(initial, Xk, beta)
        costHistory.append( costFunction(initial, Xk, beta) )
        rmseHistory.append( rmse(real, Xk) )
        # init for next iteration
        if  costFunction(initial, Xk, beta) - costFunction(initial, Xnext, beta) < finishCondition :
            break
        Xk = Xnext
    return Xnext, costHistory, rmseHistory

In [ ]:
def minimizeFast(costFunction=None, costFunctionGrad=None, \
                 real=None, initial=None, \
                 beta=0.3, learningrate=0.001, maxIter=1000, finishCondition=10):
    e = 0.1
    change = 0
    costHistory = []
    changeHistory = []
    rmseHistory = []
    Xk = np.copy(initial)
    Xnext = np.copy(initial)
    Xold = np.copy(initial)
    innerLoopCount = 0
    outerLoopCount = 0
    while True:
        costK = EPS
        costNext = 0
        n = 0
        Xk = Xold # discard last iteration
        while ( costNext< costK ):
            innerLoopCount += 1
            Xnext = Xk - (2**n) * learningrate * costFunctionGrad(initial, Xk, beta)
            costK = costFunction(initial, Xk, beta)
            costNext = costFunction(initial, Xnext, beta)
            print(n, costK, costNext, innerLoopCount)
            Xold = np.copy(Xk)
            Xk = np.copy(Xnext)
            n = n + 1
            
        costHistory.append( costFunction(initial, Xold, beta) )
        rmseHistory.append( rmse(real, Xold) )
        change = np.linalg.norm((Xk-Xold),2);
        #changeHistory.append(change)
        outerLoopCount += 1
        print (outerLoopCount, "change", change)
        if ( change < 0.01 or innerLoopCount > maxIter or abs(costK-costNext) < finishCondition ): 
            break
    return Xnext, costHistory, rmseHistory

In [ ]:

imTV, chistoryTV, rhistoryTV = minimize(
    costFunction=tvCost,
    costFunctionGrad=tvCostGrad,
    real=Xreal,
    learningrate=0.001,
    initial=Y,
    beta=0.5,
    finishCondition=1
)

In [ ]:
plt.imshow(imTV)

In [ ]:



imTVD, chistoryTVD, rhistoryTVD = minimize(
    costFunction=directionalTVCost,
    costFunctionGrad=directionalTVCostGrad,
    real=Xreal,
    learningrate=0.001,
    initial=Y,
    beta=0.5,
    finishCondition=1
)


In [ ]:
plt.imshow(imTVD)

In [ ]:

plt.figure()
plt.title("Cost Over Iterations")
plt.plot(chistoryTV)
plt.plot(chistoryTVD)
plt.figure()
plt.title("RMSE Over Iterations")
plt.plot(rhistoryTV)
plt.plot(rhistoryTVD)

In [ ]:
fig = plt.figure()

ax = fig.add_subplot(221)
ax.title.set_text('Original Image')
ax.imshow(m2i(Xreal))

ax = fig.add_subplot(222)
ax.title.set_text('Noisy Image')
ax.imshow(m2i(Y))

ax = fig.add_subplot(223)
ax.title.set_text('TV')
ax.imshow(m2i(imTV))

ax = fig.add_subplot(224)
ax.title.set_text('TV Directional')
ax.imshow(m2i(imTVD))


## 5. Results

In [ ]:
fig = plt.figure()

ax = fig.add_subplot(231)
ax.title.set_text('Original Image')
ax.imshow(m2i(Xreal))

ax = fig.add_subplot(232)
ax.title.set_text('Noisy Image')
ax.imshow(m2i(Y))

ax = fig.add_subplot(233)
ax.title.set_text('TV B=0.3')
ax.imshow(m2i(denoiseTV(Y, 0.3)))

ax = fig.add_subplot(234)
ax.title.set_text('TV B=0.7')
ax.imshow(m2i(denoiseTV(Y, 0.7)))

ax = fig.add_subplot(235)
ax.title.set_text('TV B=0.9')
ax.imshow(m2i(denoiseTV(Y, 0.9)))

ax = fig.add_subplot(236)
ax.title.set_text('TV B=1.1')
ax.imshow(m2i(denoiseTV(Y, 1.1)))

plt.show()
plt.rcParams['figure.figsize'] = [16, 10]
